In [1]:
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score, 
    log_loss, confusion_matrix, mean_squared_error, r2_score
)
import numpy as np
import pandas as pd
import os
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
# Загрузка переменных из файла .env
load_dotenv()

True

In [2]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION") # получаем ключ бакета, к которому подключён MLFlow, из .env

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 


In [3]:
# Настройки эксперимента
YOUR_NAME = "imartnv" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"churn_experiment_{YOUR_NAME}"
RUN_NAME = "model_5_registry"
REGISTRY_MODEL_NAME = "churn_model_martynov_alexey"


# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [4]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [5]:
# Пример выгрузки данных из БД
TABLE = 'alt_users_churn'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)

In [6]:
#Функция удаления дубликатов
def remove_duplicates(data):
    feature_cols = data.columns.drop('customer_id').tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data 

In [7]:
#Функция для заполнения пропусков
def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
    for col in cols_with_nans:
        if data[col].dtype in [float, int]:
            fill_value = data[col].mean()
        elif data[col].dtype == 'object':
            fill_value = data[col].mode().iloc[0]
        data[col] = data[col].fillna(fill_value)
    return data

In [8]:
#Функция удаления выбросов
def remove_outliers(df: pd.DataFrame, threshold: float = 1.5) -> pd.DataFrame:
        num_cols = df.select_dtypes(include=['float']).columns
        potential_outliers = pd.DataFrame(False, index=df.index, columns=num_cols)
        
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            margin = threshold * IQR
            lower = Q1 - margin
            upper = Q3 + margin
            potential_outliers[col] = ~df[col].between(lower, upper)
        
        outliers = potential_outliers.any(axis=1)
        df_cleaned = df[~outliers]
        return df_cleaned

In [9]:
#Почистим датасет
data = fill_missing_values(data)
data = remove_duplicates(data)
data = remove_outliers(data)

In [10]:
data = data.set_index('id')
data = data.drop(columns=['customer_id','begin_date','end_date'])

In [11]:
# 1. Разделение признаков на числовые и категориальные
num_features = data.select_dtypes(include=['float', 'int']).columns.tolist()
cat_features = data.select_dtypes(include='object').columns.tolist()

# Преобразование числовых признаков
scaler = StandardScaler()
num_features_scaled = pd.DataFrame(
    scaler.fit_transform(data[num_features]),
    columns=num_features,
    index=data.index
)

# Преобразование категориальных признаков в One-Hot Encoding
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
cat_features_encoded = pd.DataFrame(
    encoder.fit_transform(data[cat_features]),
    columns=encoder.get_feature_names_out(cat_features),
    index=data.index
)

# 2. Объединение преобразованных признаков
transformed_data = pd.concat([num_features_scaled, cat_features_encoded], axis=1)

/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# Обучение модели CatBoostClassifier
X = transformed_data.drop(columns='target')
y = data['target']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(
    iterations=100, 
    depth=4, 
    learning_rate=0.1, 
    loss_function='Logloss', 
    verbose=0
)
model.fit(X_train, y_train)

In [13]:
# Предсказания
y_pred = model.predict(X_test)  # Бинарные предсказания
y_proba = model.predict_proba(X_test)[:, 1]  # Вероятности положительного класса

In [14]:
# Метрики
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
logloss = log_loss(y_test, y_proba)

# Матрица ошибок с нормализацией
conf_matrix_normalized = confusion_matrix(y_test, y_pred, normalize='all')

# Ошибки первого и второго рода
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
err_1 = fp / (fp + tn)  # Ошибка первого рода
err_2 = fn / (fn + tp)  # Ошибка второго рода

In [15]:
# 11. Вывод метрик
print(f"ROC-AUC: {roc_auc:.2f}")
print(f"F1-мера: {f1:.2f}")
print(f"Precision (точность): {precision:.2f}")
print(f"Recall (полнота): {recall:.2f}")
print(f"Log-loss: {logloss:.2f}")
print("Confusion Matrix (normalized):")
print(conf_matrix_normalized)
print(f"Ошибка первого рода (err_1): {err_1:.2f}")
print(f"Ошибка второго рода (err_2): {err_2:.2f}")

ROC-AUC: 0.85
F1-мера: 0.58
Precision (точность): 0.71
Recall (полнота): 0.49
Log-loss: 0.42
Confusion Matrix (normalized):
[[0.66430092 0.05606813]
 [0.14194464 0.1376863 ]]
Ошибка первого рода (err_1): 0.08
Ошибка второго рода (err_2): 0.51


In [16]:
# Настройки логирования модели
# ------------------------------------------------------------------
pip_requirements = "/home/mle-user/mle_projects/mle-mlflow/requirements.txt"
signature = mlflow.models.infer_signature(X_test, y_pred)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

# Получим ID эксперимента
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

In [19]:
# Логирование в MLflow
# ------------------------------------------------------------------
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    # Логируем гиперпараметры модели
    mlflow.log_param("iterations", 100)
    mlflow.log_param("depth", 4)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("loss_function", "Logloss")
    
    # Логируем метрики
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("log_loss", logloss)
    mlflow.log_metric("error_type_1", err_1)
    mlflow.log_metric("error_type_2", err_2)

    # Пример: логируем матрицу ошибок как артефакт (картинку)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix_normalized, annot=True, fmt=".2f", cmap="Blues")
    plt.title("Normalized Confusion Matrix")
    plt.savefig("conf_matrix.png")
    plt.close()
    mlflow.log_artifact("conf_matrix.png")

    # Логируем модель (CatBoost) вместе с окружением
    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        input_example=input_example,
        metadata=metadata,
        signature=signature,
        pip_requirements=pip_requirements,
        await_registration_for=60  # время ожидания регистрации в Model Registry
    )

print("Run ID:", run_id)
print("Model logged to MLflow with ID:", model_info.model_uri)

🏃 View run model_5_registry at: http://127.0.0.1:5000/#/experiments/4/runs/8e8891064dff46409052e404dfdc9ba2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


S3UploadFailedError: Failed to upload conf_matrix.png to s3-student-mle-20241125-59b9e9f709/4/8e8891064dff46409052e404dfdc9ba2/artifacts/conf_matrix.png: An error occurred (SignatureDoesNotMatch) when calling the PutObject operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.

In [17]:
import boto3
boto3.setup_default_session(region_name='ru-central1')

In [4]:
model_uri = f"models:/{REGISTRY_MODEL_NAME}/1"  

In [5]:
# Загрузка модели из реестра
loaded_model = mlflow.catboost.load_model(model_uri)

In [19]:
model_predictions = loaded_model.predict(X_test)

In [ ]:
client = mlflow.MlflowClient()

REGISTRY_MODEL_NAME = 'churn_model_nikolaistepanov'

models = client.search_model_versions(filter_string=f"name = '{REGISTRY_MODEL_NAME}'")
print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage


print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

# поменяйте статус каждой модели
client.transition_model_version_stage(model_name_1, model_version_1, 'production')
client.transition_model_version_stage(model_name_2, model_version_2, 'staging')

# переимнуйте модель в реестре
client.rename_registered_model(name=REGISTRY_MODEL_NAME, new_name=f'{REGISTRY_MODEL_NAME}_b2c')